In [1]:
#This is the script for spatial object integration and analysis
#loading in the necessary packages
##the last line here is a workaround for memory requirements Mac
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(here)
library(tidyverse)
library(viridis)
library(RColorBrewer)
options (future.globals.maxSize = 4000 * 1024^5)

Attaching SeuratObject

Attaching sp


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


here() starts at /global/u1/k/kserrano/Desktop

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.8     ✔ purrr   0.3.5
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.3     ✔ forcats 0.5.2

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: viridisLite



In [2]:
#colors for UMAP plots
farmer_colors = c("#FF9999", "#FF6666", "#FFCC66", "#FF6699", "#CC99FF", "#CC6699", 
                  "#99CCFF", "#ff9933", "#FF99FF", "#FF99CC", "#99CCCC", "#66CC99", 
                  "#CC6600", "#FFCCCC", "#339966", "#CCCCCC", "#CC9999", "#996666", "#FFFF99")

In [2]:
#loading in pre-filtered spatial objects
B1 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/B1.rds')
A2 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/A2.rds')
B2 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/B2.rds')
C2 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/C2.rds')
D2 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/D2.rds')
A3 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/A3.rds')
B3 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/B3.rds')
C3 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/C3.rds')
D3 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/D3.rds')
A4 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/A4.rds')
B4 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/B4.rds')
C4 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/C4.rds')
D4 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/D4.rds')
A5 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/A5.rds')
B5 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/B5.rds')
C5 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/C5.rds')
D5 <- readRDS('/global/homes/k/kserrano/Desktop/Spatial_Objects/D5.rds')

#adding in original identity metadata
B1@meta.data['dataset'] = 'B1'
A2@meta.data['dataset'] = 'A2'
B2@meta.data['dataset'] = 'B2'
C2@meta.data['dataset'] = 'C2'
D2@meta.data['dataset'] = 'D2'
A3@meta.data['dataset'] = 'A3'
B3@meta.data['dataset'] = 'B3'
C3@meta.data['dataset'] = 'C3'
D3@meta.data['dataset'] = 'D3'
A4@meta.data['dataset'] = 'A4'
B4@meta.data['dataset'] = 'B4'
C4@meta.data['dataset'] = 'C4'
D4@meta.data['dataset'] = 'D4'
A5@meta.data['dataset'] = 'A5'
B5@meta.data['dataset'] = 'B5'
C5@meta.data['dataset'] = 'C5'
D5@meta.data['dataset'] = 'D5'


#assigning objects to lists for integration
#wt = all wild type capture area, both treatments
#myc = mycorrhizal treated capture areas
#ctrl = mock-inoculated capture areas
wt.list <-c(B1,A2,B2,C2,D2,A3,B3,C3,D3,A4,B4,C4,D4,A5,B5,C5,D5)
myc.list <- c(B1,A2,B2,C2,D2,A3,B3,C3,D3)
ctrl.list <- c(A4,B4,C4,D4,A5,B5,C5,D5)

In [3]:
#prepping for integration, selecting features to keep
wt.features <- SelectIntegrationFeatures(wt.list, nfeatures = 3000)
myc.features <- SelectIntegrationFeatures(myc.list, nfeatures = 3000)
ctrl.features <- SelectIntegrationFeatures(ctrl.list, nfeatures = 3000)

#getting a list of all unique genes from each list for use as anchors later
all.wt.genes <- lapply(wt.list, row.names) %>% Reduce(c, .) %>% unique()
all.myc.genes <- lapply(myc.list, row.names) %>% Reduce(c, .) %>% unique()
all.ctrl.genes <- lapply(ctrl.list, row.names) %>% Reduce(c, .) %>% unique()

#prepSCTIntegration
wt.list <-PrepSCTIntegration(wt.list, assay = NULL, anchor.features=wt.features, sct.clip.range = NULL,verbose = TRUE)
myc.list <-PrepSCTIntegration(myc.list, assay = NULL, anchor.features=myc.features, sct.clip.range = NULL,verbose = TRUE)
ctrl.list <-PrepSCTIntegration(ctrl.list, assay = NULL, anchor.features=ctrl.features, sct.clip.range = NULL,verbose = TRUE)

In [ ]:
#finding anchors for all datasets
#this takes A LONG TIME! to run, set verbose = TRUE for estimate
wt.anchors <- FindIntegrationAnchors(object.list = wt.list, normalization.method = "SCT", anchor.features = wt.features, verbose = FALSE)
saveRDS(wt.anchors, "/global/homes/k/kserrano/Desktop/Integrated_Objects/wt.anchors.rds")

ctrl.anchors <- FindIntegrationAnchors(object.list = ctrl.list, normalization.method = "SCT", anchor.features = ctrl.features, verbose = FALSE)
saveRDS(ctrl.anchors, "/global/homes/k/kserrano/Desktop/Integrated_Objects/ctrl.anchors.rds")

myc.anchors <- FindIntegrationAnchors(object.list = myc.list, normalization.method = "SCT", anchor.features = myc.features, verbose = FALSE)
saveRDS(myc.anchors, "/global/homes/k/kserrano/Desktop/Integrated_Objects/myc.anchors.rds")

In [ ]:
#this takes an even longer time to run!
#integrating each dataset individually
wt.integrated <- IntegrateData(anchorset = wt.anchors, normalization.method = "SCT", verbose =  TRUE, features.to.integrate = all.wt.genes)

In [ ]:
#this takes an even longer time to run!
#integrating each dataset individually
ctrl.integrated <- IntegrateData(anchorset = ctrl.anchors, normalization.method = "SCT", verbose = TRUE, features.to.integrate = all.ctrl.genes)

In [ ]:
#this takes an even longer time to run!
#integrating each dataset individually
myc.integrated <- IntegrateData(anchorset = myc.anchors, normalization.method = "SCT", verbose = FALSE, features.to.integrate = all.myc.genes)

In [ ]:
# specify that we will perform downstream analysis on the corrected data note that the
# original unmodified data still resides in the 'Spatial' assay
DefaultAssay(wt.integrated) <- "integrated"
# Run the standard workflow for visualization and clustering, then save the integrated object!
wt.integrated <- RunPCA(wt.integrated, npcs = 30, verbose = FALSE)
wt.integrated <- RunUMAP(wt.integrated, reduction = "pca",verbose = FALSE, dims = 1:30, metric ="cosine")
wt.integrated <- FindNeighbors(wt.integrated, reduction = "pca", dims = 1:30)
wt.integrated <- FindClusters(wt.integrated, resolution = 0.5)
saveRDS(wt.integrated, "/global/homes/k/kserrano/Desktop/Integrated_Objects/wt.integrated.rds")

In [ ]:
# Run the standard workflow for visualization and clustering, then save the integrated object!
DefaultAssay(ctrl.integrated) <- "integrated"
ctrl.integrated <- RunPCA(ctrl.integrated, npcs = 30, verbose = FALSE)
ctrl.integrated <- RunUMAP(ctrl.integrated, reduction = "pca",verbose = FALSE, dims = 1:30, metric ="cosine")
ctrl.integrated <- FindNeighbors(ctrl.integrated, reduction = "pca", dims = 1:30)
ctrl.integrated<- FindClusters(ctrl.integrated, resolution = 0.5)
saveRDS(ctrl.integrated, "/global/homes/k/kserrano/Desktop/Integrated_Objects/ctrl.integrated.rds")

In [ ]:
# Run the standard workflow for visualization and clustering, then save the integrated object!
DefaultAssay(myc.integrated) <- "integrated"
# Run the standard workflow for visualization and clustering
myc.integrated <- RunPCA(myc.integrated, npcs = 30, verbose = FALSE)
myc.integrated <- RunUMAP(myc.integrated, reduction = "pca",verbose = FALSE, dims = 1:30, metric ="cosine")
myc.integrated <- FindNeighbors(myc.integrated, reduction = "pca", dims = 1:30)
myc.integrated <- FindClusters(myc.integrated, resolution = 0.5)
saveRDS(myc.integrated, "/global/homes/k/kserrano/Desktop/Integrated_Objects/myc.integrated.rds")

In [ ]:
#Code for Figure S4 - a
#UMAP plot of all datasets integrated, + grouped by dataset
p00 <- DimPlot(wt.integrated, reduction = "umap", cols = farmer_colors)
#Code for Figure S4 - b
p01 <- DimPlot(wt.integrated, reduction = "umap", group.by = "dataset", cols = farmer_colors)
p00+p01

In [ ]:
#UMAP plot of all control datasets integrated,  + grouped by dataset
p02 <- DimPlot(ctrl.integrated, reduction = "umap", cols = farmer_colors)
p03 <- DimPlot(ctrl.integrated, reduction = "umap", group.by = "dataset", cols = farmer_colors)
p02+p03

In [ ]:
#UMAP plot of all mycorrhizal datasets integrated, + grouped by dataset
p04 <- DimPlot(myc.integrated, reduction = "umap", cols = farmer_colors)
p05 <- DimPlot(myc.integrated, reduction = "umap", group.by = "dataset", cols = farmer_colors)
p04+p05